In [1]:
import numpy as np

from sympy import (Matrix,
                   sin,
                   cos,
                   symbols,
                   diff,
                   simplify,
                   Eq,
                   Function,
                   lambdify,
                   Symbol,
                   Function,
                   Derivative,
                   matrix2numpy,
                   linear_eq_to_matrix)
from sympy.physics.mechanics import dynamicsymbols
from sympy import default_sort_key 

# Dynamic System Derivation

In [2]:
C = lambda th: Matrix([[cos(th), -sin(th)], [sin(th), cos(th)]])
D = lambda th: Matrix([[sin(th), cos(th)], [-cos(th), sin(th)]])

In [3]:
t = symbols('t')

# Define states and their time derivatives
th_s, th_1, th_2, th_t = dynamicsymbols('theta_s theta_1 theta_2 theta_T')
dth_s, dth_1, dth_2, dth_t = dynamicsymbols('theta_s theta_1 theta_2, theta_T', 1)
ddth_s, ddth_1, ddth_2, ddth_t = dynamicsymbols('theta_s theta_1 theta_2, theta_T', 2)

# Generalized coordinates
q = Matrix([th_s, th_1, th_2, th_t])
qdot = Matrix([dth_s, dth_1, dth_2, dth_t])
qddot = Matrix([ddth_s, ddth_1, ddth_2, ddth_t])

# State vector
y = q.row_insert(4, qdot)
ydot = qdot.row_insert(4, qddot)

# Controls
tau_s = symbols('tau_s')
tau_1 = symbols('tau_1')
tau_2 = symbols('tau_2')
tau_t = symbols('tau_t')
tau = Matrix([tau_s, tau_1, tau_2, tau_t])

# System parameters
rho = symbols('rho_s rho_1 rho_2 rho_t')    # dimensional array
m = symbols('m_s m_1 m_2 m_t')              # mass array
I = symbols('I_s I_1 I_2 I_t')              # inertia array
d = symbols('d_s d_1 d_2 d_t')              # disturbance vector

# Satellite central position
r_s0, r_s1 = symbols('r_s0 r_s1')
r_s = Matrix([r_s0, r_s1])

# Target central position
r_t0, r_t1 = symbols('r_s0 r_s1')
r_t = Matrix([r_t0, r_t1])

# Mid-link positions
r_1 = r_s + C(y[0])@(Matrix([rho[0], 0.]) + C(y[1])@Matrix([rho[1], 0.]))
r_2 = r_1 + C(y[0])@C(y[1])@(Matrix([rho[1], 0.]) + C(y[2])@Matrix([rho[2], 0.]))

# Link velocities
v_1 = Matrix(diff(r_1, t))
v_2 = Matrix(diff(r_2, t))

# Define Lagrangian
T = 0.5*m[1]*v_1.dot(v_1) + 0.5*m[2]*v_2.dot(v_2) + \
    0.5*I[0]*y[4]**2 + 0.5*I[1]*y[5]**2 + \
    0.5*I[2]*y[6]**2
V = 0.

L = simplify(T - V)

In [4]:
dL_dth_s = diff(L, y[0])
dL_dth_1 = diff(L, y[1])
dL_dth_2 = diff(L, y[2])

dL_ddth_s = diff(L, y[4])
dL_ddth_1 = diff(L, y[5])
dL_ddth_2 = diff(L, y[6])

dL_ddth_s_dt = diff(dL_ddth_s, t)
dL_ddth_1_dt = diff(dL_ddth_1, t)
dL_ddth_2_dt = diff(dL_ddth_2, t)

th_s_eqn = Eq(simplify(dL_ddth_s_dt - dL_dth_s), tau[0] - d[0])
th_1_eqn = Eq(simplify(dL_ddth_1_dt - dL_dth_1), tau[1] - d[1])
th_2_eqn = Eq(simplify(dL_ddth_2_dt - dL_dth_2), tau[2] - d[2])
th_t_eqn = Eq(I[3]*qddot[3], tau[3] - d[3])

In [5]:
eqs = [th_s_eqn.lhs - th_s_eqn.rhs,
       th_1_eqn.lhs - th_1_eqn.rhs,
       th_2_eqn.lhs - th_2_eqn.rhs,
       th_t_eqn.lhs - th_t_eqn.rhs]

A, B = linear_eq_to_matrix(eqs, qddot[0], qddot[1], qddot[2], qddot[3])
dynamics_func = lambdify((y, tau, r_s, r_t, rho, m, I, d), A.LUsolve(B), 'numpy')

TODO: Find a good way to keep this in matrix form but still solve for tau.

In [6]:
th_s_eqn = Eq(tau[0], simplify(dL_ddth_s_dt - dL_dth_s) + d[0])
th_1_eqn = Eq(tau[1], simplify(dL_ddth_1_dt - dL_dth_1) + d[1])
th_2_eqn = Eq(tau[2], simplify(dL_ddth_2_dt - dL_dth_2) + d[2])
th_t_eqn = Eq(tau[3], I[3]*qddot[3] + d[3])

vec = Matrix([th_s_eqn.rhs,
              th_1_eqn.rhs,
              th_2_eqn.rhs,
              th_t_eqn.rhs])

subs = {qddot[0] : symbols('ddth_s', real=True),
        qddot[1] : symbols('ddth_1', real=True),
        qddot[2] : symbols('ddth_2', real=True),
        qddot[3] : symbols('ddth_t', real=True)}

vec = vec.subs(subs)

inverse_dynamics_func = lambdify((y, list(subs.values()), r_s, r_t, rho, m, I, d), vec, 'numpy')

# Other Important Parameters
- [x] end effector position
- [x] end effector velocity
- [x] Jacobian of end effector
- [x] Jacobian of end eddfector's time derivative
- [x] critical point positions
- [x] critical point velocities
- [x] capture point position
- [x] capture point velocity
- [x] capture point acceleration
- [x] obstacle point positions
- [x] obstacle point velocities
- [x] $\gamma_1$
- [x] $\gamma_2$
- [x] $\gamma$
- [x] $\mathbf{\lambda}$
- [x] $\mathbf{\dot{\lambda}}$

In [7]:
# End effector position
r_ee = r_1 + C(y[0])@C(y[1])@(Matrix([rho[1], 0.]) + C(y[2])@Matrix([2.*rho[2], 0.]))

# End effector velocity
v_ee = diff(r_ee, t)

# End effector jacobian
J = r_ee.jacobian(q[:-1])

# End effector
J_dot = simplify(diff(J, t))

In [8]:
r_ee_func = lambdify((y, rho, r_s), matrix2numpy(r_ee), 'numpy')
v_ee_func = lambdify((y, rho, r_s), v_ee, 'numpy')
J_func = lambdify((y, rho, r_s), J, 'numpy')
J_dot_func = lambdify((y, rho, r_s), J_dot, 'numpy')

In [9]:
# Critical point positions
cp_1 = r_s + C(y[0])@(Matrix([rho[0], 0.]) + C(y[1])@Matrix([2.*rho[1], 0.]))
cp_2 = r_2
cp_3 = r_ee

# Critical point velocities
v_cp_1 = diff(cp_1, t)
v_cp_2 = v_2
v_cp_3 = v_ee

In [10]:
cp_func = lambdify((y, rho, r_s), (cp_1, cp_2, cp_3), 'numpy')
v_cp_func = lambdify((y, rho, r_s), (v_cp_1, v_cp_2, v_cp_3), 'numpy')

In [11]:
# Capture point position
r_c = r_t + C(y[3])@Matrix([0., rho[3]])

# Capture point velocity
v_c = Matrix(diff(r_c, t))

# Capture point acceleration
a_c = diff(v_c, t)

sub_dict = {ddth_t: symbols('theta_ddot_t', real=True)}
a_c = a_c.subs(sub_dict)

In [12]:
r_c_func = lambdify((y, rho, r_t), r_c, 'numpy')
v_c_func = lambdify((y, rho, r_t), v_c, 'numpy')
a_c_func = lambdify((y, sub_dict[ddth_t], rho, r_t), a_c, 'numpy')

In [13]:
# Obstacle point positions
op_1 = r_t + C(y[3])@Matrix([-rho[3], 0.])
op_2 = r_t + C(y[3])@Matrix([-rho[3], rho[3]])
op_3 = r_t + C(y[3])@Matrix([rho[3], rho[3]])
op_4 = r_t + C(y[3])@Matrix([rho[3], 0.])
op_5 = r_t + C(y[3])@Matrix([rho[3], -rho[3]])
op_6 = r_t + C(y[3])@Matrix([0., -rho[3]])
op_7 = r_t + C(y[3])@Matrix([-rho[3], -rho[3]])

# Obstacle point velocities
v_op_1 = diff(op_1, t)
v_op_2 = diff(op_2, t)
v_op_3 = diff(op_3, t)
v_op_4 = diff(op_4, t)
v_op_5 = diff(op_5, t)
v_op_6 = diff(op_6, t)
v_op_7 = diff(op_7, t)

In [14]:
ops_func = lambdify((y, rho, r_t), (op_1, op_2, op_3, op_4, op_5, op_6, op_7), 'numpy')
v_ops_func = lambdify((y, rho, r_t), (v_op_1, v_op_2, v_op_3, v_op_4, v_op_5, v_op_6, v_op_7), 'numpy')

In [15]:
# Gamma 1
p = symbols('p_1 p_2 p_3')
q_bar = symbols('q_bar_1 q_bar_2 q_bar_3')
q_max = symbols('q_max_1 q_max_2 q_max_3')
q_min = symbols('q_min_1 q_min_2 q_min_3')

gam_1 = (1./3.)*(p[0]*((q[0] - q_bar[0])/(q_max[0] - q_min[0]))**2 + \
                 p[1]*((q[1] - q_bar[1])/(q_max[1] - q_min[1]))**2 + \
                 p[2]*((q[2] - q_bar[2])/(q_max[2] - q_min[2]))**2)

In [16]:
gam_1_func = lambdify((y, p, q_bar, q_max, q_min), gam_1, 'numpy')

In [17]:
# Gamma 2
cp_x = [cp_1[0], cp_2[0], cp_3[0]]
cp_y = [cp_1[1], cp_2[1], cp_3[1]]

op_x = [[Function(f"xT_{j+1}_{k+1}")(t) for k in range(2)] for j in range(3)]
op_y = [[Function(f"yT_{j+1}_{k+1}")(t) for k in range(2)] for j in range(3)]

w11, w12, w21, w22, w31, w32 = symbols('w11 w12 w21 w22 w31 w32', real=True)
weights = {(0, 0): w11, (0, 1): w12,
           (1, 0): w21, (1, 1): w22,
           (2, 0): w31, (2, 1): w32}

gam_2 = 0
for j in range(3):
    for k in range(2):
        d_jk_sq = (cp_x[j] - op_x[j][k])**2 + (cp_y[j] - op_y[j][k])**2
        gam_2 += weights[(j, k)] / d_jk_sq

In [18]:
w = Matrix([w11, w12, w21, w22, w31, w32])
gam_2_func = lambdify((y, rho, r_s, w, op_x, op_y), gam_2, 'numpy')

In [19]:
gam = Matrix([gam_1 + gam_2])

In [20]:
gam_func = lambdify((y, rho, r_s, p, q_bar, q_max, q_min, w, op_x, op_y), gam, 'numpy')

In [21]:
lam = gam.jacobian(q[:-1])

In [22]:
lam_func = lambdify((y, rho, r_s, p, q_bar, q_max, q_min, w, op_x, op_y), lam, 'numpy')

In [23]:
lam_dot = diff(lam, t)

In [24]:
subs_dict = {}
velocity_symbols = {}  # To store the order of our velocity symbols if needed.
for j in range(3):
    for k in range(2):
        vx_sym = symbols(f"vxT_{j+1}_{k+1}", real=True)
        vy_sym = symbols(f"vyT_{j+1}_{k+1}", real=True)
        velocity_symbols[(j, k, 'x')] = vx_sym
        velocity_symbols[(j, k, 'y')] = vy_sym
        subs_dict[diff(op_x[j][k], t)] = vx_sym
        subs_dict[diff(op_y[j][k], t)] = vy_sym

lam_dot = lam_dot.subs(subs_dict)

In [25]:
vxT = Matrix(list(subs_dict.values())[::2])
vyT = Matrix(list(subs_dict.values())[1::2])

In [26]:
lam_dot_func = lambdify((y, rho, r_s, p, q_bar, q_max, q_min, w, op_x, op_y, vxT, vyT), lam_dot, 'numpy')

# Testing Functions

In [27]:
y = np.zeros(8)
u = np.zeros(4)
u[1] = np.deg2rad(30.)

r_s = np.array([-1.5, 0.])
r_t = np.array([1.5, 0.])

rho = np.array([0.5, 0.5, 0.5, 0.5])
m = np.array([250., 25., 25., 180.])
I = np.array([25., 2.5, 2.5, 18.])
d = np.zeros(4)

p = np.ones(3)
w = np.ones(6)
q_bar = np.zeros(3)
q_max = np.ones(3)*np.pi
q_min = np.ones(3)*-np.pi

In [28]:
dynamics_func(y, u, r_s, r_t, rho, m, I, d)

array([[-0.02223101],
       [ 0.05148234],
       [-0.04680213],
       [ 0.        ]])

In [29]:
r_ee_func(y, rho, r_s)

array([[1.],
       [0.]])

In [30]:
v_ee_func(y, rho, r_s)

array([[0.],
       [0.]])

In [31]:
J_func(y, rho, r_s)

array([[-0. , -0. , -0. ],
       [ 2.5,  2. ,  1. ]])

In [32]:
J_dot_func(y, rho, r_s)

array([[-0., -0., -0.],
       [-0., -0., -0.]])

In [33]:
cp_func(y, rho, r_s)

(array([[0.],
        [0.]]),
 array([[0.5],
        [0. ]]),
 array([[1.],
        [0.]]))

In [34]:
v_cp_func(y, rho, r_s)

(array([[-0.],
        [ 0.]]),
 array([[0.],
        [0.]]),
 array([[0.],
        [0.]]))

In [35]:
r_c_func(y, rho, r_t)

array([[1.5],
       [0.5]])

In [36]:
v_c_func(y, rho, r_t)

array([[-0.],
       [-0.]])

In [37]:
a_c_func(y, 0., rho, r_t)

array([[ 0.],
       [-0.]])

In [38]:
ops_func(y, rho, r_t)

(array([[1.],
        [0.]]),
 array([[1. ],
        [0.5]]),
 array([[2. ],
        [0.5]]),
 array([[2.],
        [0.]]),
 array([[ 2. ],
        [-0.5]]),
 array([[ 1.5],
        [-0.5]]),
 array([[ 1. ],
        [-0.5]]))

In [39]:
v_ops_func(y, rho, r_t)

(array([[ 0.],
        [-0.]]),
 array([[ 0.],
        [-0.]]),
 array([[-0.],
        [ 0.]]),
 array([[-0.],
        [ 0.]]),
 array([[0.],
        [0.]]),
 array([[0.],
        [0.]]),
 array([[0.],
        [0.]]))

In [40]:
gam_1_func(y, p, q_bar, q_max, q_min)

np.float64(0.0)

In [41]:
# add method for finding closest 2 points to each CP
ops = list(ops_func(y, rho, r_t))
cpt = r_c_func(y, rho, r_t)
ops += [cpt]
ops = [op.flatten() for op in ops]

cps = list(cp_func(y, rho, r_s))
cps = [cp.flatten() for cp in cps]

inds = np.zeros((3, 2), dtype=int)

pos = np.zeros((3, 2, 2))
for i, cp in enumerate(cps):
    dists = [np.sqrt((cp[0] - op[0])**2 + \
                            (cp[1] - op[1])**2) for op in ops]
    sorted_dists = sorted(dists, key=float)

    ind1 = dists.index(sorted_dists[0])
    ind2 = dists.index(sorted_dists[1])
    
    if i == 2 and ind1 == len(ops)-1:
        ind1 = dists.index(sorted_dists[2])

    if i == 2 and ind2 == len(ops)-1:
        ind2 = dists.index(sorted_dists[2])

    pos[i, 0, :] = ops[ind1]
    pos[i, 1, :] = ops[ind2]
    inds[i, 0] = ind1
    inds[i, 1] = ind2

print(gam_2_func(y, rho, r_s, w, pos[:, 0, :], pos[:, 1, :]))

9.100000000000001


In [42]:
gam_func(y, rho, r_s, p, q_bar, q_max, q_min, w, pos[:, 0, :], pos[:, 1, :])

array([[9.1]])

In [43]:
lam_func(y, rho, r_s, p, q_bar, q_max, q_min, w, pos[:, 0, :], pos[:, 1, :])

array([[41.91, 29.7 ,  5.28]])

In [44]:
v_ops = list(v_ops_func(y, rho, r_t))
v_cpt = v_c_func(y, rho, r_t)
v_ops += [v_cpt]
v_ops = [v.flatten() for v in v_ops]

vs = np.array([[[v_ops[ind1][0], v_ops[ind1][1]], [v_ops[ind2][0], v_ops[ind2][1]]] for ind1, ind2 in inds])

lam_dot_func(y, rho, r_s, p, q_bar, q_max, q_min, w, pos[:, 0, :], pos[:, 1, :], vs[:, :, 0].flatten(), vs[:, :, 0].flatten())

array([[0., 0., 0.]])

In [45]:
inverse_dynamics_func(y, np.ones(4), r_s, r_t, rho, m, I, d)

array([[262.5 ],
       [171.25],
       [ 52.5 ],
       [ 18.  ]])

# Save Functions in PKL

In [46]:
import dill

with open('functions/r_ee.pkl', 'wb') as f:
    dill.dump(r_ee_func, f)

with open('functions/v_ee.pkl', 'wb') as f:
    dill.dump(v_ee_func, f)

with open('functions/J.pkl', 'wb') as f:
    dill.dump(J_func, f)
    
with open('functions/J_dot.pkl', 'wb') as f:
    dill.dump(J_dot_func, f)

with open('functions/cp.pkl', 'wb') as f:
    dill.dump(cp_func, f)
    
with open('functions/v_cp.pkl', 'wb') as f:
    dill.dump(v_cp_func, f)

with open('functions/r_c.pkl', 'wb') as f:
    dill.dump(r_c_func, f)

with open('functions/v_c.pkl', 'wb') as f:
    dill.dump(v_c_func, f)

with open('functions/a_c.pkl', 'wb') as f:
    dill.dump(a_c_func, f)

with open('functions/ops.pkl', 'wb') as f:
    dill.dump(ops_func, f)

with open('functions/v_ops.pkl', 'wb') as f:
    dill.dump(v_ops_func, f)

with open('functions/gam1.pkl', 'wb') as f:
    dill.dump(gam_1_func, f)

with open('functions/gam2.pkl', 'wb') as f:
    dill.dump(gam_2_func, f)

with open('functions/gam.pkl', 'wb') as f:
    dill.dump(gam_func, f)

with open('functions/lam.pkl', 'wb') as f:
    dill.dump(lam_func, f)

with open('functions/lam_dot.pkl', 'wb') as f:
    dill.dump(lam_dot_func, f)

with open('functions/inverse_dynamics.pkl', 'wb') as f:
    dill.dump(inverse_dynamics_func, f)

with open('functions/dynamics.pkl', 'wb') as f:
    dill.dump(dynamics_func, f)